# SC-Conv model

In [1]:
import torch

In [2]:
import argparse
from torchsummary import summary
import tqdm
from tqdm import tqdm_notebook as tq
import os, time, math, copy ,random
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from collections import namedtuple
import matplotlib.pyplot as plt

torch.set_printoptions(precision=8, linewidth=50000)
import warnings
warnings.filterwarnings(action='ignore')


/usr/local/lib/python3.6/dist-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=16, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=7, help='Number of bit width')
args = parser.parse_args(args=[])

## Definition

In [4]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

In [5]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

In [6]:
def snum(a):
    if a >= 0 :
        return '0'
    else :
        return '1'

In [7]:
def binInv(bIn):
    bOut = bin(int(bIn,2)^(pow(2,len(bIn))-1)).replace('0b','').rjust(len(bIn),'0')
    return bOut

In [8]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [9]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [10]:
def flp2fixTensor(fIn, iBW, iBWF):
    fMin = - 2 ** (iBW - iBWF - 1)
    fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
    fList = []
    for aTensor in fIn.view(-1):
        fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
    return torch.tensor(fList).view(fIn.size())

In [11]:
class LFSR7:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        self.b4 = eval(f'str(random.randint(0,1))')
        self.b5 = eval(f'str(random.randint(0,1))')
        self.b6 = eval(f'str(random.randint(0,1))')
        if int(self.b0) + int(self.b1) + int(self.b2) + int(self.b3) + int(self.b4) + int(self.b5) + int(self.b6) == 0 :
            self.b0 = eval(f'str(random.randint(0,1))')
            self.b1 = eval(f'str(random.randint(0,1))')
            self.b2 = eval(f'str(random.randint(0,1))')
            self.b3 = eval(f'str(random.randint(0,1))')
            self.b4 = eval(f'str(random.randint(0,1))')
            self.b5 = eval(f'str(random.randint(0,1))')
            self.b6 = eval(f'str(random.randint(0,1))')
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[5]),int(stream[6]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        self.b4 = stream[3]
        self.b5 = stream[4]
        self.b6 = stream[5]
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        self.b4 = '0'
        self.b5 = '0'
        self.b6 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6

In [12]:
def LFSRlist7():
    lfsr = LFSR7()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

In [13]:
def Comp(a,lfsr,snum):
    for com in range(0,len(a)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

In [14]:
def perm(a):
    al = len(a)
    blist = []
    if args.frac_bits == args.bBW : 
        for i in range(al) :
            blist.append(a[al-i-1])
    elif args.frac_bits > args.bBW :
        for i in range(al-(args.frac_bits-args.bBW)) :
            blist.append((a[al-(args.frac_bits-args.bBW)-i-1]))
    b = "".join(blist)
    b = b + ('0'*(args.frac_bits-args.bBW))
    return b

In [15]:
def findMaxMin(data):
    max = torch.max(data)
    min = torch.min(data)
    SF=torch.max(abs(max),abs(min)).item()
    return SF

In [16]:
def CountOne(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        n = 0
        for a in num:
            if a == '1' :
                n += 1
        if num[0] == '1' :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.shape)

In [17]:
def defSign(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        if num[0] == '1' :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.shape)

## SC-MAC

### SNG

In [18]:
def SNG(iIN,lfsr):
    if iIN == 1:
        iIN = 0.9999
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    if sNUM == 1 :
        bFRAC = bin(int(binInv(bFRAC),2)+1).replace('0b','').rjust(args.bBW,'0')
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if bIN == args.full_bits*'0' :
        return '0'*(2**(args.bBW)+1)
    else :
        return sA

In [19]:
def SngNumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG(float(aNumpy),lfsr))
                     
    return np.array(sList).reshape(fIn.size())

In [20]:
def Sngp(iIN,lfsr):
    if iIN == 1:
        iIN = 0.9999
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    if sNUM == 1 :
        bFRAC = bin(int(binInv(bFRAC),2)+1).replace('0b','').rjust(args.bBW,'0')
    oAlist = []
    for k in range(2**(args.bBW)): #lfsr number generating    
        lNUM = perm(lfsr[k])
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    
    if bIN == args.full_bits*'0' :
        return '0'*(2**(args.bBW)+1)
    else :
        return sA

In [21]:
def SngpNumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(Sngp(float(aNumpy),lfsr))
    return  np.array(sList).reshape(fIn.size())

### Multiplier

In [22]:
def mul(a,b):
    al = len(a)
    bl = len(b)
    
    outlist = []
    
    if al != bl :
        print("length of string is different")
        return 0
    
    outlist.append(XOR(a[0],b[0]))
    
    for i in range(al-1) :
        outlist.append(str(int(a[i+1]) & int(b[i+1])))
    
    #print(outlist)
    out = "".join(outlist)
    
    return out    

In [23]:
def MulNumpy(aIn,wIn):
    oList = []
    for a,w in zip(aIn.reshape(-1),wIn.reshape(-1)):
        oList.append(mul(a,w))
    return np.array(oList)

### Converter

In [24]:
def Convert(sIn,SF):
    s = defSign(sIn)
    o = (CountOne(sIn)/(2**args.bBW))*SF*s
    return o

## sc-mul module

In [25]:
def SCmulTensor(aIn,wIn,aSF,wSF):
    a = aIn/aSF
    w = wIn/wSF
    
    lfsr = LFSRlist7()
    
    Sa = SngNumpy(a,lfsr)
    Sw = SngpNumpy(w,lfsr)

    
    
    Smul = MulNumpy(Sa,Sw)
    
    Bout = Convert(Smul,aSF*wSF)
    
    return Bout

### TEST

In [35]:
act = torch.rand((6,1,32,32))-torch.randint(5,(6,1,32,32))+torch.randint(5,(6,1,32,32))
wei = torch.rand((1,1,5,5))-torch.randint(5,(1,1,5,5))+torch.randint(5,(1,1,5,5))

In [36]:
aSF = findMaxMin(act)
wSF = findMaxMin(wei)

In [37]:
out = SCmulTensor(act,wei,aSF,wSF)

## SC-Conv

In [38]:
def conv(X, filters, bias, stride=1, pad=0):
    n, c, h, w = X.shape # 1, 1, 32, 32
    n_f, _, filter_h, filter_w = filters.shape
    out_h = (h+2*pad-filter_h)//stride + 1
    out_w = (w+2*pad-filter_w)//stride + 1
    # add padding to height and width.
    in_X = F.pad(X,(0,0,0,0,pad,pad,pad,pad),"constant", 0)
    out  = torch.zeros((n, n_f, out_h, out_w))
    
    for i in range(n): # for each image.
        aSF = findMaxMin(in_X[i])
        for c in range(n_f): # for each channel.
            bSF = findMaxMin(filters[c])
            for h in range(out_h): # slide the filter vertically.
                h_start = h * stride
                h_end = h_start + filter_h
                for w in range(out_w): # slide the filter horizontally.
                    w_start = w * stride
                    w_end = w_start + filter_w
                    # Element-wise multiplication.
                    out[i, c, h, w] = torch.sum(in_X[i,:,h_start:h_end,w_start:w_end]*filters[c]) + bias[c]
    
    return out

In [39]:
c = conv(act, wei, (0,0), stride=1, pad=0)

In [40]:
def StoConv(X, filters, bias, stride=1, pad=0):
    n, c, h, w = X.shape # 1, 1, 32, 32
    n_f, _, filter_h, filter_w = filters.shape

    out_h = (h+2*pad-filter_h)//stride + 1
    out_w = (w+2*pad-filter_w)//stride + 1
    # add padding to height and width.
    in_X = F.pad(X,(0,0,0,0,pad,pad,pad,pad),"constant", 0)
    out  = torch.zeros((n, n_f, out_h, out_w))
    
    for i in range(n): # for each image.
        aSF = findMaxMin(in_X[i])
        for c in range(n_f): # for each channel.
            bSF = findMaxMin(filters[c])
            for h in range(out_h): # slide the filter vertically.
                h_start = h * stride
                h_end = h_start + filter_h
                for w in range(out_w): # slide the filter horizontally.
                    w_start = w * stride
                    w_end = w_start + filter_w
                    # Element-wise multiplication.
                    out[i, c, h, w] = torch.sum(SCmulTensor(in_X[i,:,h_start:h_end,w_start:w_end],filters[c],aSF,wSF)) + bias[c]
    
    return out

In [41]:
s = StoConv(act, wei, (0,0), stride=1, pad=0)

In [42]:
print(s)

tensor([[[[ 3.37134132e+01,  2.03685226e+01,  2.87968750e+01,  ..., -1.33448925e+01,  1.40472574e+01, -1.17645769e+01],
          [ 2.10708828e+01, -8.07717323e+00,  1.42228479e+01,  ...,  2.01929302e+01,  1.00086718e+01, -1.14133949e+01],
          [ 5.47842979e+01, -4.21417665e+00,  3.16063309e+00,  ...,  3.58205032e+01, -4.91654015e+00,  3.44157753e+01],
          ...,
          [ 1.94905663e+01, -2.01929302e+01,  1.59787521e+01,  ...,  1.70322990e+01,  7.02362061e-01,  1.75590687e+01],
          [ 1.22913599e+00,  1.80858383e+01,  3.31866379e+01,  ..., -4.74094677e+00, -1.21157579e+01,  9.13071632e+00],
          [ 3.51181030e-01,  3.82787704e+01, -1.98417473e+01,  ..., -9.48189735e+00,  4.46000366e+01,  3.95079041e+01]]],


        [[[ 5.42900038e+00,  1.83885460e+01,  2.04900932e+01,  ..., -3.76527367e+01,  4.22060890e+01,  1.06828690e+01],
          [-3.85283947e+00,  3.39750290e+01,  3.32745075e+00,  ..., -1.57615852e+00,  2.57439613e+01, -8.05593300e+00],
          [ 2.4693187

In [43]:
print(c)

tensor([[[[ 2.96106339e+01,  1.61081638e+01,  2.51126690e+01,  ..., -1.70837860e+01,  1.10050697e+01, -1.55635862e+01],
          [ 1.71955109e+01, -1.16305113e+01,  1.04683723e+01,  ...,  1.62587795e+01,  6.62597561e+00, -1.56415749e+01],
          [ 5.05896225e+01, -7.79678726e+00, -7.27064133e-01,  ...,  3.30547142e+01, -8.40707016e+00,  3.09404850e+01],
          ...,
          [ 1.54782114e+01, -2.32445984e+01,  1.18737183e+01,  ...,  1.31983509e+01, -3.58203602e+00,  1.43653889e+01],
          [-2.77177048e+00,  1.41450386e+01,  2.97059555e+01,  ..., -8.55652809e+00, -1.64323101e+01,  6.05891705e+00],
          [-3.36800385e+00,  3.49712982e+01, -2.39026413e+01,  ..., -1.34600811e+01,  4.11933784e+01,  3.61611977e+01]]],


        [[[ 9.88167763e-01,  1.36560631e+01,  1.59670439e+01,  ..., -4.19963722e+01,  3.90770149e+01,  6.64798260e+00],
          [-8.00060558e+00,  3.03267326e+01, -1.43518066e+00,  ..., -5.36063004e+00,  2.18933048e+01, -1.19238358e+01],
          [ 2.0611005